# Gallery Example: Par/M/1 Queue

This example demonstrates a Pareto/M/1 queueing system:
- **Arrivals**: Pareto distribution (heavy-tailed)
- **Service**: Exponential
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Pareto inter-arrival times create bursty arrival patterns with occasional long gaps.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_parm1(): Network {
    """Create Par/M/1 queueing model"""
    val model = Network("Par/M/1")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
// Pareto arrivals with mean=1 and SCV=64 (very heavy tail)
    source.setArrival(oclass1, Pareto.fit_mean_and_scv(1, 64))
    queue.setService(oclass1, Exp(2))
// Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_parm1()

## About Pareto Arrival Times

Pareto inter-arrival times create:
- Periods of rapid arrivals (many jobs in short time)
- Followed by long quiet periods (occasional very long gaps)
- Self-similar traffic patterns common in network modeling

With SCV = 64:
- Very high arrival variability
- Bursty traffic pattern
- Queue may build up during bursts even at low average utilization

In [ ]:
// Solve with Fluid solver (CTMC may struggle with heavy tails)
println("\n=== Solver Results ===")
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)